<a href="https://colab.research.google.com/github/doma27/phonegap/blob/master/templates/aistudio_gemini_prompt_freeform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [2]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

### Install & import

In [3]:
!pip install -U -q google-generativeai

In [4]:
# Install the client library and import necessary modules.
import google.generativeai as genai

import base64
import copy
import hashlib
import io
import json
import mimetypes
import pathlib
import pprint
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

## Set the API key

Add your API_KEY to the secrets manager in the left panel "🔑".

In [1]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")

In [ ]:
# Configure the client library by providing your API key.
genai.configure(api_key=GOOGLE_API_KEY)

## Parse the arguments

In [5]:
model = 'gemini-1.5-pro-002' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6Im1vZGVsID0gZ2VuYWkuR2VuZXJhdGl2ZU1vZGVsKCdtb2RlbHMvZ2VtaW5pLTEuNS1wcm8tMDAyJylcXG5yZXNwb25zZSA9IG1vZGVsLmdlbmVyYXRlX2NvbnRlbnQoY29udGVudHM9XFxcIldobyB3b24gV2ltYmxlZG9uIHRoaXMgeWVhcj9cXFwiLFxcbiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICB0b29scz0nZ29vZ2xlX3NlYXJjaF9yZXRyaWV2YWwnKVxcbnByaW50KHJlc3BvbnNlKSJ9XX0seyJyb2xlIjoibW9kZWwiLCJwYXJ0cyI6W3sidGV4dCI6IllvdSdyZSB0cnlpbmcgdG8gdXNlIHRoZSBgZ2VuYWlgIGxpYnJhcnkgdG8gZ2V0IHRoZSBXaW1ibGVkb24gd2lubmVyIHVzaW5nIEdvb2dsZSBTZWFyY2guIEhvd2V2ZXIsIHRoZSBjb2RlIHNuaXBwZXQgeW91IHByb3ZpZGVkIGlzIGluY29tcGxldGUgYW5kIG5lZWRzIHNvbWUgYWRqdXN0bWVudHMgdG8gZnVuY3Rpb24gY29ycmVjdGx5LiBIZXJlJ3MgaG93IHlvdSBjYW4gZml4IGl0IGFuZCBhY2hpZXZlIHRoZSBkZXNpcmVkIHJlc3VsdDpcXG5cXG5gYGBweXRob25cXG5pbXBvcnQgZ2VuYWlcXG5cXG4jIEFzc3VtaW5nIHlvdSBoYXZlIHRoZSAnZ2VuYWknIGxpYnJhcnkgaW5zdGFsbGVkXFxubW9kZWwgPSBnZW5haS5HZW5lcmF0aXZlTW9kZWwoJ21vZGVscy9nZW1pbmktMS41LXByby0wMDInKVxcblxcbiMgRGVmaW5lIHRoZSBwcm9tcHRcXG5wcm9tcHQgPSBcXFwiV2hvIHdvbiBXaW1ibGVkb24gdGhpcyB5ZWFyP1xcXCJcXG5cXG4jIFVzZSB0aGUgJ2dvb2dsZV9zZWFyY2hfcmV0cmlldmFsJyB0b29sXFxucmVzcG9uc2UgPSBtb2RlbC5nZW5lcmF0ZV9jb250ZW50KFxcbiAgICBjb250ZW50cz1wcm9tcHQsXFxuICAgIHRvb2xzPSdnb29nbGVfc2VhcmNoX3JldHJpZXZhbCdcXG4pXFxuXFxuIyBQcmludCB0aGUgZXh0cmFjdGVkIGFuc3dlclxcbnByaW50KHJlc3BvbnNlKVxcbmBgYFxcblxcbioqRXhwbGFuYXRpb246KipcXG5cXG4xLiAqKkltcG9ydCB0aGUgYGdlbmFpYCBMaWJyYXJ5OioqIEVuc3VyZSB5b3UgaGF2ZSB0aGUgYGdlbmFpYCBsaWJyYXJ5IGluc3RhbGxlZC4gSWYgbm90LCB1c2UgYHBpcCBpbnN0YWxsIGdlbmFpYC5cXG4yLiAqKkNyZWF0ZSBhIEdlbmVyYXRpdmUgTW9kZWw6KiogWW91IGluaXRpYWxpemUgYSBgR2VuZXJhdGl2ZU1vZGVsYCBvYmplY3Qgd2l0aCB0aGUgZGVzaXJlZCBtb2RlbC4gXFxuMy4gKipEZWZpbmUgdGhlIFByb21wdDoqKiBDbGVhcmx5IHN0YXRlIHlvdXIgcXVlc3Rpb24gaW4gYSBzdHJpbmcgdmFyaWFibGUuXFxuNC4gKipVc2UgdGhlIGBnb29nbGVfc2VhcmNoX3JldHJpZXZhbGAgVG9vbDoqKiAgVGhlIGB0b29sc2AgYXJndW1lbnQgc3BlY2lmaWVzIHRoYXQgeW91IHdhbnQgdG8gdXNlIEdvb2dsZSBTZWFyY2ggZm9yIHJldHJpZXZhbC5cXG41LiAqKkdlbmVyYXRlIENvbnRlbnQ6KiogIENhbGwgdGhlIGBnZW5lcmF0ZV9jb250ZW50YCBtZXRob2QgdG8gdHJpZ2dlciB0aGUgbW9kZWwgYW5kIHJldHJpZXZhbCBwcm9jZXNzLlxcbjYuICoqUHJpbnQgdGhlIFJlc3BvbnNlOioqIFRoZSBgcmVzcG9uc2VgIG9iamVjdCB3aWxsIGNvbnRhaW4gdGhlIGV4dHJhY3RlZCBhbnN3ZXIgZnJvbSBHb29nbGUgU2VhcmNoLCB3aGljaCB5b3UgY2FuIHRoZW4gcHJpbnQuXFxuXFxuKipJbXBvcnRhbnQgQ29uc2lkZXJhdGlvbnM6KipcXG5cXG4qICoqQXV0aGVudGljYXRpb246KiogWW91IG1pZ2h0IG5lZWQgdG8gc2V0IHVwIGF1dGhlbnRpY2F0aW9uIHdpdGggR29vZ2xlIFNlYXJjaCBBUEkgZm9yIHRoaXMgdG8gd29yayBwcm9wZXJseS4gUmVmZXIgdG8gdGhlIGBnZW5haWAgZG9jdW1lbnRhdGlvbiBmb3IgZ3VpZGFuY2UuXFxuKiAqKk1vZGVsIEF2YWlsYWJpbGl0eToqKiBFbnN1cmUgeW91IGhhdmUgYWNjZXNzIHRvIHRoZSBgZ2VtaW5pLTEuNS1wcm8tMDAyYCBtb2RlbCBvciBhIGNvbXBhdGlibGUgYWx0ZXJuYXRpdmUuXFxuKiAqKkFQSSBLZXlzOioqICBJZiB5b3UgYXJlIHVzaW5nIHRoZSBgZ2VuYWlgIGxpYnJhcnksIHlvdSB3aWxsIG5lZWQgdG8gc2V0IHVwIEFQSSBrZXlzIHRvIGFjY2VzcyB0aGUgR2VtaW5pIG1vZGVsLiAgVGhlICdnZW5haScgZG9jdW1lbnRhdGlvbiB3aWxsIHByb3ZpZGUgZGV0YWlscyBvbiBob3cgdG8gb2J0YWluIGFuZCBzZXQgdXAgdGhlIG5lY2Vzc2FyeSBrZXlzLlxcblxcbkJ5IG1ha2luZyB0aGVzZSBhZGp1c3RtZW50cywgeW91IHNob3VsZCBiZSBhYmxlIHRvIHF1ZXJ5IEdvb2dsZSBTZWFyY2ggYW5kIHJldHJpZXZlIHRoZSBXaW1ibGVkb24gd2lubmVyIGZyb20gdGhlIG1vc3QgcmVsZXZhbnQgcmVzdWx0cy5cXG4ifV19XQ==' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MS4xNSwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

In [6]:
gais_contents

[{'role': 'user',
  'parts': [{'text': 'model = genai.GenerativeModel(\'models/gemini-1.5-pro-002\')\\nresponse = model.generate_content(contents=\\"Who won Wimbledon this year?\\",\\n                                  tools=\'google_search_retrieval\')\\nprint(response)'}]},
 {'role': 'model',
  'parts': [{'text': 'You\'re trying to use the `genai` library to get the Wimbledon winner using Google Search. However, the code snippet you provided is incomplete and needs some adjustments to function correctly. Here\'s how you can fix it and achieve the desired result:\\n\\n```python\\nimport genai\\n\\n# Assuming you have the \'genai\' library installed\\nmodel = genai.GenerativeModel(\'models/gemini-1.5-pro-002\')\\n\\n# Define the prompt\\nprompt = \\"Who won Wimbledon this year?\\"\\n\\n# Use the \'google_search_retrieval\' tool\\nresponse = model.generate_content(\\n    contents=prompt,\\n    tools=\'google_search_retrieval\'\\n)\\n\\n# Print the extracted answer\\nprint(response)\\n```

In [7]:
generation_config

{'temperature': 1.15, 'top_p': 0.95, 'top_k': 40, 'max_output_tokens': 8192}

In [8]:
safety_settings

{}

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [ ]:
# @title `show_file` function
drive = None
def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")

In [ ]:
for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

## Convert & upload files

For each file, Google AI Studio either sent:
- a Google Drive IDs
- a URL, or
- the raw bytes (`inline_data`).

The API itself onlty understands two ways of sending files:

- `inline_data` - where the bytes are placed inline in the request.
- `file_data` - where the file is uploaded to the Files API, and you pass a reference to that file.

This section goes through the `contents` from Google AI Studio, and uploads the file data, to the Files API, so Gemini can access it:

In [ ]:
# @title `upload_file` function

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)

drive = None
def upload_file_data(file_data):
    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        hash = hashlib.sha256(data).hexdigest()
        path = tempfiles / hash
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files"in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

In [ ]:
contents = copy.deepcopy(gais_contents)

for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data)

Here is the coverted `Content`s:

In [ ]:
contents

## Call `generate_content`

In [ ]:
# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=False,
)

In [ ]:
if generation_config.get("candidate_count", 1) == 1:
    display(Markdown(response.text))

In [ ]:
response.candidates

## Or Create a chat

In [ ]:
gemini = genai.GenerativeModel(
    model_name=model,
    generation_config=generation_config,
    safety_settings=safety_settings,
)

A `ChatSession`, should always end with the model's turn.

So before creating the `chat` check whos turn was last.

If the user was last, attach all but the last content as the `history` and send the last content with `send_message` to get the model's response.

If the model was last, put the whole contents list in the history.

In [ ]:
model_turn = contents[-1].get("role", None) == "user"

if model_turn:
    chat = gemini.start_chat(history=contents[:-1])

    response = chat.send_message(contents[-1])

    if generation_config.get("candidate_count", 1) == 1:
        display(Markdown(response.text))
else:
    chat = gemini.start_chat(history=contents)

In [ ]:
if model_turn:
    response.candidates

After that use `send_message` to continue the conversation.

In [ ]:
# response = chat.send_message("Interesting, tell me more.")
# display(Markdown(response.text))